 This notebook is responsible for aggregating the data set into one large dataframe/parquet

In [2]:
from pyspark.sql import SparkSession, functions as F
import findspark
findspark.init()
import pandas as pd

Read in all the data and group it accordingly

In [3]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.parquet.enableVectorizedReader", False)
    .getOrCreate()
)

22/08/02 12:14:04 WARN Utils: Your hostname, Sens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.12.253.187 instead (on interface en0)
22/08/02 12:14:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/02 12:14:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sdf_taxi = spark.read.parquet('../data/raw/taxi_data/*.parquet')

In [34]:
sdf_weather = spark.read.option("header", True).csv('../data/raw/weather_data.csv')
sdf_weather = sdf_weather.withColumn(
        'DATE',
        F.col('DATE').cast('date'))
sdf_weather = sdf_weather.filter((F.year(sdf_weather.DATE) == 2017) | (F.year(sdf_weather.DATE) == 2018))

Reduce dataframes to only the desired attributes

In [35]:
sdf_weather = sdf_weather.select("DATE", "PRCP", "SNOW", "TMAX", "TMIN")

In [12]:
sdf_taxi = sdf_taxi.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "payment_type", "trip_distance", "PULocationID", "DOLocationID", "fare_amount")

In [36]:
sdf_weather.show(1, vertical = True)

-RECORD 0----------
 DATE | 2017-01-01 
 PRCP |     0      
 SNOW |     0      
 TMAX |    89      
 TMIN |    44      
only showing top 1 row



In [13]:
sdf_taxi.show(1, vertical = True)

-RECORD 0------------------------------------
 tpep_pickup_datetime  | 2017-01-01 11:32:05 
 tpep_dropoff_datetime | 2017-01-01 11:37:48 
 passenger_count       | 1                   
 payment_type          | 2                   
 trip_distance         | 1.2                 
 PULocationID          | 140                 
 DOLocationID          | 236                 
 fare_amount           | 6.5                 
only showing top 1 row



In [37]:
sdf = sdf_taxi.join(sdf_weather, F.to_date("tpep_pickup_datetime") == F.col("DATE"), "inner")

In [38]:
sdf.show(1, vertical = True)

-RECORD 0------------------------------------
 tpep_pickup_datetime  | 2017-01-01 11:32:05 
 tpep_dropoff_datetime | 2017-01-01 11:37:48 
 passenger_count       | 1                   
 payment_type          | 2                   
 trip_distance         | 1.2                 
 PULocationID          | 140                 
 DOLocationID          | 236                 
 fare_amount           | 6.5                 
 DATE                  | 2017-01-01          
 PRCP                  |     0               
 SNOW                  |     0               
 TMAX                  |    89               
 TMIN                  |    44               
only showing top 1 row

